<a href="https://colab.research.google.com/github/GeoffreySaxena/Comparision-of-Various-Algorithms-for-Cross-Domain-Sentiment-Analysis/blob/main/MAJOR_1_Comparision_of_Various_Algorithms_for_Cross_Domain_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/drive/MyDrive/Cross-domain-sentiment-analysis-master/

/content/drive/MyDrive/Cross-domain-sentiment-analysis-master


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
import os
import re
import sys
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt  

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn import svm
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,confusion_matrix
from sklearn.feature_selection import SelectKBest,chi2
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
train_list = ["Dataset/Actualdata/Books/Bookstrain.txt","Dataset/Actualdata/Dvd/Dvdtrain.txt","Dataset/Actualdata/Electronics/Electronicstrain.txt","Dataset/Actualdata/Kitchen/Kitchentrain.txt"]

test_list = ["Dataset/Actualdata/Books/Bookstest.txt","Dataset/Actualdata/Dvd/Dvdtest.txt","Dataset/Actualdata/Electronics/Electronicstest.txt","Dataset/Actualdata/Kitchen/Kitchentest.txt"]

In [5]:
stopword = stopwords.words('english') 

In [6]:
def preprocess(sentence):
	sentence = re.sub('[^\w\s]'," ",str(sentence))
	sentence = re.sub('[^a-zA-Z]'," ",str(sentence))
	sents = word_tokenize(sentence)
	new_sents = " "
	for i in range(len(sents)):
		if(sents[i].lower() not in stopword):
			new_sents+=sents[i].lower()+" "
	return new_sents

In [7]:
def preprocess_test(choice):

	the_file = test_list[choice]
	#print(the_file)
	with open(the_file,'r',encoding='utf-8') as f:
		test_data = f.readlines()

	corpus_test = []

	for i in range(400):
		sent = test_data[i]
		sent = sent[0:len(sent)-1]
		corpus_test.append(preprocess(sent))

	#print(corpus_test[0])

	label_test = np.zeros(400)
	label_test[0:200] = 1


	return [corpus_test,label_test]

In [8]:
def preprocess_train(choice):


	the_file = train_list[choice]
	#print(the_file)
	with open(the_file,'r',encoding='utf-8') as f:
		train_data = f.readlines()


	corpus_train = []

	for i in range(1600):
		sent = train_data[i]
		sent = sent[0:len(sent)-1]
		corpus_train.append(preprocess(sent))

	#print(corpus_train[0])

	label_train = np.zeros(1600)
	label_train[0:800] = 1

	return [corpus_train,label_train]

In [9]:
def preprocessing(train_choice,test_choice):

	corpus_train,label_train = preprocess_train(train_choice)

	corpus_test,label_test = preprocess_test(test_choice)

	return corpus_train,label_train,corpus_test,label_test

In [10]:
def extract_files(pos_file,neg_file,num):
	
	with open(pos_file,'r',encoding='utf-8') as f:
		pos = f.readlines()
	with open(neg_file,'r',encoding='utf-8') as f:
		neg = f.readlines()


	corpus_pos = []
	flag = False

	for i in range(len(pos)):
		if(pos[i]=="<review_text>\n"):
			flag = True
			new_str = " "
			continue
		elif(pos[i]=="</review_text>\n"):
			flag = False
			corpus_pos.append(new_str)
			continue
		if(flag):
			if(len(pos[i])>1):
				sent = pos[i]
				sent = sent[0:len(sent)-1]
				if(sent[0]=='\t'):
					sent = sent[1:len(sent)-1]
			new_str+=sent  
	

	corpus_neg = []
	flag = False


	for i in range(len(neg)):
		if(neg[i]=="<review_text>\n"):
			flag = True
			new_str = " "
			continue
		elif(neg[i]=="</review_text>\n"):
			flag = False
			corpus_neg.append(new_str)
			continue
		if(flag):
			if(len(neg[i])>1):
				sent = neg[i]
				sent = sent[0:len(sent)-1]
				if(sent[0]=='\t'):
					sent = sent[1:len(sent)-1]
			new_str+=sent

	
	train = []

	for i in range(800):
		train.append(corpus_pos[i])

	for i in range(800):
		train.append(corpus_neg[i])


	test = []

	for i in range(800,1000):
		test.append(corpus_pos[i])

	for i in range(800,1000):
		test.append(corpus_neg[i])

	
	dir_to_write =["Books","Dvd","Electronics","Kitchen"]
	original_dir = "Dataset/Actualdata"+"/"+dir_to_write[num]+"/"

	train_file = original_dir+dir_to_write[num]+"train.txt"
	test_file = original_dir+dir_to_write[num]+"test.txt"


	
	with open(train_file,'w',encoding='utf-8') as f:
		for i in range(len(train)):
			f.write(train[i])
			f.write("\n")


	with open(test_file,'w',encoding='utf-8') as f:
		for i in range(len(test)):
			f.write(test[i])
			f.write("\n")

In [11]:
vector_parameters = [[2,0.8],[3,0.8],[3,0.8],[3,0.8],[3,0.8],[3,0.8],[3,0.8],[3,0.8],[3,0.8],
[3,0.8],[3,0.8],[3,0.8]]

In [12]:
def featureextraction(train_corpus,test_corpus,label_train,train_choice,test_choice):

	val = (train_choice)*3 + test_choice

	param  = vector_parameters[val]
	mindf = param[0]
	maxdf = param[1]

	vectorizer = TfidfVectorizer(min_df=mindf,max_df=maxdf,use_idf=True,sublinear_tf=True,stop_words='english')

	train_corpus_tf_idf = vectorizer.fit_transform(train_corpus,label_train)

	test_corpus_tf_idf = vectorizer.transform(test_corpus)

	return [train_corpus_tf_idf,test_corpus_tf_idf]

In [13]:
chi_square_parameters = [[5000],[4000],[4000],[4000],[4000],[4000],[4000],['all'],['all'],['all'],
['all'],['all'],[2500],['all']]

In [14]:
def featureselection(train_corpus_tf_idf,test_corpus_tf_idf,label_train,train_choice,test_choice):

	val = (train_choice)*3 + test_choice

	k = chi_square_parameters[val][0]

	if(k=='all'):
		K = train_corpus_tf_idf.shape[1]
	else:
		K = k 

	vectorizer_chi2 = SelectKBest(chi2,k=K)

	chi_train_corpus_tf_idf = vectorizer_chi2.fit_transform(train_corpus_tf_idf,label_train)

	chi_test_corpus_tf_idf = vectorizer_chi2.transform(test_corpus_tf_idf)

	return [chi_train_corpus_tf_idf,chi_test_corpus_tf_idf]

In [15]:
lr_parameters = [[7,0.5],[8,0.5],[6,0.5],[10,0.8],[8,0.6],[14,0.9],[7,0.8],[8,0.8],[3,0.8],[5,0.7],[2,0.5],[5,0.9]]
rbf_parameters = [[8,0.8],[9,0.6],[13,0.7],[5,0.6],[7,0.5],[5,0.6],[5,0.6],[12,0.8],[2,0.6],[5,0.8],[8,0.8],[5,0.8]]
dt_parameters = [[8,0.8],[9,0.5],[15,0.5],[15,0.9],[8,0.9],[10,0.7],[15,0.9],[25,0.8],[15,0.9],[5,0.9],[15,0.9],[15,0.9]]
linear_parameters = [[8,0.5],[10,0.5],[7,0.9],[10,0.5],[13,0.6],[5,0.5],[7,0.8],[9,0.9],[3,0.4],[5,0.8],[5,0.9],[5,0.8]]
nb_parameters = [[8,0.5],[3,0.6],[5,0.7],[2,0.5],[5,0.9],[5,0.9],[3,0.9],[8,0.8],[5,0.9],[11,0.6],[3,0.9],[3,0.9]]
knn_parameters = [[7,0.5],[7,0.6],[7,0.5],[9,0.6],[15,0.8],[15,0.9],[10,0.8],[10,0.8],[15,0.9],[5,0.8],[10,0.9],[10,0.8]]

In [16]:
def bagging_classify_sentiment(classifier,chi_train_corpus_tf_idf,chi_test_corpus_tf_idf,label_train,label_test,num,samples):
    
    clf   = BaggingClassifier(base_estimator=classifier,random_state=0,max_samples=samples,n_estimators=num)
    clf.fit(chi_train_corpus_tf_idf,label_train)
    pred = clf.predict(chi_test_corpus_tf_idf)
    accuracy = clf.score(chi_test_corpus_tf_idf,label_test)
    cm = confusion_matrix(pred,label_test)
    f1 = f1_score(pred,label_test)
    return accuracy,f1,cm 

In [17]:
def bagging_train(chi_train_corpus_tf_idf,label_train,chi_test_corpus_tf_idf,label_test,train_choice,test_choice):

	x = (train_choice)*3 + test_choice

	rbf_function = [[0.9],[0.9],[0.9],[0.9],[0.9],[0.8],[0.9],[0.9],[0.8],[0.9],[0.9],[0.9]]

	Gamma = rbf_function[x][0]

	classify = ["LR","SVM-RBF","DT","SVM-L","MNB","KNN"]



	classifiers = [LogisticRegression(random_state=0),SVC(gamma=Gamma),DecisionTreeClassifier(random_state=0),SVC(kernel='linear',gamma=Gamma),MultinomialNB(),KNeighborsClassifier(n_neighbors=7)]

	param = [lr_parameters[x],rbf_parameters[x],dt_parameters[x],linear_parameters[x],nb_parameters[x],knn_parameters[x]]

	for i in range(len(classifiers)):
		p = param[i]
		print(p)
		num = p[0]
		samples = p[1]
		acc,f1,cm = bagging_classify_sentiment(classifiers[i],chi_train_corpus_tf_idf,chi_test_corpus_tf_idf,label_train,label_test,num,samples)


		
		print("Bagging "+classify[i]+" "+"F1 score is :",f1)
		print("Bagging "+classify[i]+" "+"confusion matrix is:")
		print(cm)
		print("\n")

In [18]:
def classify_sentiment(classifier,chi_train_corpus_tf_idf,chi_test_corpus_tf_idf,label_train,label_test):
    clf   = classifier
    clf.fit(chi_train_corpus_tf_idf,label_train)
    pred = clf.predict(chi_test_corpus_tf_idf)
    accuracy = clf.score(chi_test_corpus_tf_idf,label_test)
    cm = confusion_matrix(pred,label_test)
    f1 = f1_score(pred,label_test)
    return accuracy,f1,cm 

In [19]:
def classifier_train(chi_train_corpus_tf_idf,label_train,chi_test_corpus_tf_idf,label_test,train_choice,test_choice):

    rbf_parameters = [[0.9],[0.9],[0.9],[0.9],[0.9],[0.8],[0.9],[0.9],[0.8],[0.9],[0.9],[0.9]]

    val = (train_choice)*3 + test_choice

    Gamma = rbf_parameters[val][0]

    classifiers = [LogisticRegression(random_state=0),SVC(gamma=Gamma),DecisionTreeClassifier(random_state=0),
                   SVC(kernel='linear',gamma=Gamma),MultinomialNB(),KNeighborsClassifier(n_neighbors=7)]

    
    accu = []
    
    classify = ["LR","SVM-RBF","DT","SVM-L","MNB","KNN"]

    for i in range(len(classifiers)):
        acc,f1,cm = classify_sentiment(classifiers[i],chi_train_corpus_tf_idf,chi_test_corpus_tf_idf,label_train,label_test)

        accu.append(acc)

        print(classify[i]+" "+"F1 score is :",f1)
        print(classify[i]+" "+"confusion matrix is:")
        print(cm)
        print("\n")

In [20]:
def getvalue(val):
	if(val=="books"):
		return 0
	elif(val=="dvd"):
		return 1
	elif(val=="electronics"):
		return 2
	else:
		return 3

In [21]:
def getchoices(train,test):

	if(train=="books"):
		train_choice = 0
		if(test=="dvd"):
			test_choice = 0
		elif(test=="electronics"):
			test_choice = 1
		else:
			test_choice = 2

	if(train=="dvd"):
		train_choice = 1
		if(test=="books"):
			test_choice = 0
		elif(test=="electronics"):
			test_choice = 1
		else:
			test_choice = 2

	if(train=="electronics"):
		train_choice = 2
		if(test=="books"):
			test_choice = 0
		elif(test=="dvd"):
			test_choice = 1
		else:
			test_choice = 2

	if(train=="kitchen"):
		train_choice = 3
		if(test=="books"):
			test_choice = 0
		elif(test=="dvd"):
			test_choice = 1
		else:
			test_choice = 2

	return train_choice,test_choice

In [23]:
original_dir = "Dataset/sorted_data_acl"

data_list = ["/books","/dvd","/electronics","/kitchen_and_housewares"]

In [24]:
for i in range(4):
	dir_path = original_dir+data_list[i]

	
	pos_file = dir_path+"/positive.review"
	neg_file = dir_path+"/negative.review"

	extract_files(pos_file,neg_file,i)

In [25]:
if __name__ == "__main__":

	train = input("Enter dataset name to train: ")

	test = input("Enter dataset name to test: ")

	
	train_choice,test_choice = getchoices(train,test)

	pre_choice_train = getvalue(train)
	pre_choice_test = getvalue(test)

	corpus_train,label_train,corpus_test,label_test = preprocessing(pre_choice_train,pre_choice_test)

	
	train_corpus_tf_idf,test_corpus_tf_idf = featureextraction(corpus_train,corpus_test,label_train,train_choice,test_choice)

	chi_train_corpus_tf_idf,chi_test_corpus_tf_idf = featureselection(train_corpus_tf_idf,test_corpus_tf_idf,label_train,train_choice,test_choice)

	classifier_train(chi_train_corpus_tf_idf,label_train,chi_test_corpus_tf_idf,label_test,train_choice,test_choice)

	bagging_train(chi_train_corpus_tf_idf,label_train,chi_test_corpus_tf_idf,label_test,train_choice,test_choice)

Enter dataset name to train: books
Enter dataset name to test: kitchen
LR F1 score is : 0.7281553398058253
LR confusion matrix is:
[[138  50]
 [ 62 150]]


SVM-RBF F1 score is : 0.7245657568238213
SVM-RBF confusion matrix is:
[[143  54]
 [ 57 146]]


DT F1 score is : 0.624
DT confusion matrix is:
[[142  83]
 [ 58 117]]


SVM-L F1 score is : 0.6835443037974683
SVM-L confusion matrix is:
[[140  65]
 [ 60 135]]


MNB F1 score is : 0.6852791878172588
MNB confusion matrix is:
[[141  65]
 [ 59 135]]


KNN F1 score is : 0.6245210727969349
KNN confusion matrix is:
[[ 41  37]
 [159 163]]


[6, 0.5]
Bagging LR F1 score is : 0.7380952380952381
Bagging LR confusion matrix is:
[[135  45]
 [ 65 155]]


[13, 0.7]
Bagging SVM-RBF F1 score is : 0.7164179104477613
Bagging SVM-RBF confusion matrix is:
[[142  56]
 [ 58 144]]


[15, 0.5]
Bagging DT F1 score is : 0.690807799442897
Bagging DT confusion matrix is:
[[165  76]
 [ 35 124]]


[7, 0.9]
Bagging SVM-L F1 score is : 0.725925925925926
Bagging SVM-L co